In [1]:
!pip install fuzzywuzzy -q
!git clone https://github.com/castorini/BuboQA.git
%cd /content/BuboQA
!bash setup.sh 

Cloning into 'BuboQA'...
remote: Enumerating objects: 1383, done.
remote: Total 1383 (delta 0), reused 0 (delta 0), pack-reused 1383
Receiving objects: 100% (1383/1383), 361.07 KiB | 6.69 MiB/s, done.
Resolving deltas: 100% (908/908), done.
/content/BuboQA
Running fetch_dataset script

--2021-10-15 13:58:02--  https://git.uwaterloo.ca/jimmylin/BuboQA-data/raw/master/SimpleQuestions_v2.tgz
Resolving git.uwaterloo.ca (git.uwaterloo.ca)... 129.97.83.4
Connecting to git.uwaterloo.ca (git.uwaterloo.ca)|129.97.83.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 423435590 (404M) [application/x-gzip]
Saving to: ‘SimpleQuestions_v2.tgz’

SimpleQuestions_v2. 100%[===================>] 403.82M  28.7MB/s    in 12s     

2021-10-15 13:58:18 (34.9 MB/s) - ‘SimpleQuestions_v2.tgz’ saved [423435590/423435590]



Unzipping SimpleQuestions dataset...

tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Igno

In [4]:
cp -r /content/BuboQA/indexes /content/drive/MyDrive

In [1]:
import pickle
import pandas as pd
from tqdm import tqdm

In [2]:
# mapping between MIDs and names in the form of dict['MID']=['str1', 'str2', ...,  'strN']
with open('/content/drive/MyDrive/indexes/names_2M.pkl', 'rb') as f:
    mid2name = pickle.load(f)

In [3]:
name = 'fb:m.0367ty'
mid2name['fb:m.05zppz']

['dude', 'guy', 'male', 'man', 'men', 'boy']

In [4]:
# In/Out degree for each MID in the form of dict['MID']=[In degree, Out degree]
with open('/content/drive/MyDrive/indexes/degrees_2M.pkl', 'rb') as f:
    degrees_2M = pickle.load(f)

In [5]:
# reverse mapping between string and MID in the form of dict['string']=[('MID', 'actual string', 'freebase type') ...] 
with open('/content/drive/MyDrive/indexes/entity_2M.pkl', 'rb') as f:
    entity_2M = pickle.load(f)

In [6]:
# mapping between MIDs and Relations in the form of dict['MID']=[{'fb:common.topic.notable_types', 'fb:people.person.gender', 'fb:people.person.profession'}]
with open('/content/drive/MyDrive/indexes/reachability_2M.pkl', 'rb') as f:
    reachability_2M = pickle.load(f)

In [7]:
reverb2freebace = pd.read_csv('/content/drive/MyDrive/reverb2freebase.csv')
reverb2freebace['freebase_ID_argument1'] = reverb2freebace['freebase_ID_argument1'].apply(lambda string:'fb:m.'+string)

In [8]:
mid_count = len(mid2name)
string_count = sum([len(name) for name in mid2name.values()])
original_relations = sum([len(relation) for relation in reachability_2M.values()])

matched_mids = 0
new_entity_strings = 0
new_relations = 0

for index, row in tqdm(reverb2freebace.iterrows(), total=reverb2freebace.shape[0]):
  mid = row['freebase_ID_argument1']
  reverb_string = row['arg1']
  relation = row['rel']
  try:
    temp = mid2name[mid]
    matched_mids+=1
  except:
    continue
  if reverb_string not in temp:
    temp.append(reverb_string)
    new_entity_strings+=1
  temp = reachability_2M[mid]
  if relation not in temp:
    temp.add(relation)
    new_relations+=1
  try:
    temp = entity_2M[reverb_string]
    temp.add((mid, relation, None))
  except:
    entity_2M[reverb_string] = set([(mid, relation, None)])
  
  degrees_2M[mid][1]+=1


print(f'\nOriginal MIDs: {mid_count}\tMatched MIDs: {matched_mids}')
print(f'Original Entity Strings: {string_count}\tAdded Entity Strings: {new_entity_strings}') 
print(f'Original Relations Count: {original_relations}\tAdded Relations Count: {new_relations}') 

100%|██████████| 153354/153354 [00:16<00:00, 9240.01it/s]


Original MIDs: 1951909	Matched MIDs: 144572
Original Entity Strings: 2995306	Added Entity Strings: 89624
Original Relations Count: 7188598	Added Relations Count: 137938


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/804 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/461 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]